In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.NoiseModel import construct_noise_model_QuantinuumH1_1
from LogicalQ.Benchmarks import n_qubit_ghz_generation
from LogicalQ.Experiments import benchmark_noise, circuit_scaling_experiment, noise_scaling_experiment, qec_cycle_efficiency_experiment
from LogicalQ.Analysis import circuit_scaling_bar3d

In [ ]:
# We use the Steane code to demonstrate the library
steane_stabilizer_tableau = [
  "XXXXIII",
  "IXXIXXI",
  "IIXXIXX",
  "ZZZZIII",
  "IZZIZZI",
  "IIZZIZZ",
]

## Circuit scaling

In [ ]:
# Construct circuit factories as circuit inputs
physical_circuit_factory = n_qubit_ghz_generation

def log_n_qubit_ghz_generation(n_qubits=3, circuit_length=None, barriers=False):
    pqc = n_qubit_ghz_generation(n_qubits, circuit_length, barriers=barriers)
    lqc = LogicalCircuit.from_physical_circuit(pqc, label=(7,1,3), stabilizer_tableau=steane_stabilizer_tableau)
    lqc.measure_all(with_error_correction=False)
    return lqc

logical_circuit_factory = log_n_qubit_ghz_generation

# Construct noise model factories as noise model inputs
noise_model_factory = construct_noise_model_QuantinuumH1_1

In [ ]:
phys_data = circuit_scaling_experiment(
    physical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=64,
    shots=128,
    with_mp=True
)

In [ ]:
log_data = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=1,
    shots=128,
    with_mp=True
)

In [ ]:
circuit_scaling_bar3d(phys_data)

In [ ]:
circuit_scaling_bar3d(log_data)

## Noise model scaling

## QEC cycle efficiency